In [ ]:
# based on https://github.com/yunjey/pytorch-tutorial/tree/master/tutorials/01-basics/feedforward_neural_network

In [2]:
%pip install torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 54.1 MB/s eta 0:00:00:00:010:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch
import torch.nn as nn
import torchvision # torchvision.datasets, torchvision.transforms
import torchvision.transforms as transforms

In [7]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [8]:
# Hyper-parameters
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [11]:
# MNIST dataset
train_dataset = torchvision.datasets.MNIST(
    root='./data',
    train=True,
    transform=transforms.ToTensor(),
    download=True,
)
test_dataset = torchvision.datasets.MNIST(
    root='./data',
    train=False,
    transform=transforms.ToTensor(),
)

100%|██████████| 9912422/9912422 [00:00<00:00, 156337281.32it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 56420910.03it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 138823144.18it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 11580868.55it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [13]:
# Data Loader
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
)
test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=False,
)

In [14]:
# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super(NeuralNet,self).__init__()
        self.fc1 = nn.Linear(input_size,hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size,output_size)

    def forward(self,x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [15]:
model = NeuralNet(input_size,hidden_size,num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    model.train()
    for i, (images,labels) in enumerate(train_loader):
        # move tensors to the configured device
        images = images.reshape(-1,28*28).to(device)
        labels = labels.to(device)

        # forward pass
        outputs = model(images)
        loss = criterion(outputs,labels)

        # backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_step}], Loss: {loss.item():.4f}')

# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images,labels in test_loader:
        images = images.reshape(-1,28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data,1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network on the 10000 test images: {100*correct/total} %')

Epoch [1/5], Step [100/600], Loss: 0.3645
Epoch [1/5], Step [200/600], Loss: 0.3318
Epoch [1/5], Step [300/600], Loss: 0.2617
Epoch [1/5], Step [400/600], Loss: 0.1763
Epoch [1/5], Step [500/600], Loss: 0.1743
Epoch [1/5], Step [600/600], Loss: 0.0599
Epoch [2/5], Step [100/600], Loss: 0.1599
Epoch [2/5], Step [200/600], Loss: 0.1396
Epoch [2/5], Step [300/600], Loss: 0.1447
Epoch [2/5], Step [400/600], Loss: 0.0798
Epoch [2/5], Step [500/600], Loss: 0.0681
Epoch [2/5], Step [600/600], Loss: 0.0768
Epoch [3/5], Step [100/600], Loss: 0.0988
Epoch [3/5], Step [200/600], Loss: 0.0363
Epoch [3/5], Step [300/600], Loss: 0.0533
Epoch [3/5], Step [400/600], Loss: 0.0619
Epoch [3/5], Step [500/600], Loss: 0.0552
Epoch [3/5], Step [600/600], Loss: 0.0165
Epoch [4/5], Step [100/600], Loss: 0.0396
Epoch [4/5], Step [200/600], Loss: 0.0314
Epoch [4/5], Step [300/600], Loss: 0.0770
Epoch [4/5], Step [400/600], Loss: 0.0139
Epoch [4/5], Step [500/600], Loss: 0.0987
Epoch [4/5], Step [600/600], Loss: